In [9]:
from pathlib import Path
import os
import csv

In [10]:
input_dir = Path("/mnt/ss5acq/SmartSPIM_Data_test/2025_07_21/20250721_16_52_12_SySy_Test_StartStop_5")

metadata_path = input_dir / "metadata.txt"

In [15]:
def pair_key_value_lists(keys, values):
    # utility function for building metadata dict

    d = {}
    for i in range(0, len(keys)):
        key = keys[i]
        val = values[i]
        if key != '':
            d[key] = val
    return d

def get_target_number(z_block, z_step):
    # Calculates number of images per tile
    try:
        steps_per_tile = max(math.ceil(float(z_block) / float(z_step)) - 1, 1)
    except:
        steps_per_tile = 1
    return steps_per_tile

def parse_destripe_tag(tag):
    status = 'true'
    for i in 'NCDA':
        if i in tag:
            status = 'false'
            tag = tag.replace(i, '')
    return status, tag

def get_metadata_v5(metadata_path):
    # builds metadata dict

    metadata_dict = {
        'channels': [],
        'tiles': []
    }
    sections = {
        'channel_vals': [],
        'tile_vals': []
    }
    with open(metadata_path, encoding="utf8", errors="ignore") as f:
        reader = csv.reader(f, dialect='excel', delimiter='\t')
        section_num = 0
        for row in reader:
            if section_num == 0:
                sections['gen_keys'] = row
                section_num += 1
                continue
            if section_num == 1:
                sections['gen_vals'] = row
                section_num += 1
                continue
            if section_num == 2:
                sections['channel_keys'] = row
                section_num += 1
                continue
            if section_num == 3:
                if row[0] != 'X':
                    sections['channel_vals'].append(row)
                    continue
                else:
                    sections['tile_keys'] = row
                    section_num += 2
                    continue
            if section_num == 5:
                sections['tile_vals'].append(row)

    d = pair_key_value_lists(sections['gen_keys'], sections['gen_vals'])
    target_number = get_target_number(d['Z_Block'], d['Z step (m)'])
    d['destripe_status'], d['destripe'] = parse_destripe_tag(d['Destripe'])
    metadata_dict.update({'sample metadata': d})

    for channel in sections['channel_vals']:
        d = pair_key_value_lists(sections['channel_keys'], channel)
        metadata_dict['channels'].append(d)

    for tile in sections['tile_vals']:
        d = pair_key_value_lists(sections['tile_keys'], tile)
        d['NumImages'] = 1
        if int(d['Skip']) == 1:
            d['NumImages'] = target_number
        metadata_dict['tiles'].append(d)
    return metadata_dict

In [16]:
a = get_metadata_v5(metadata_path)

In [17]:
a

{'channels': [{'Wavelength': '445', 'Power L': '20.00', 'Power R': '20.00'},
  {'Wavelength': '488', 'Power L': '20.00', 'Power R': '20.00'},
  {'Wavelength': '561', 'Power L': '40.00', 'Power R': '40.00'},
  {'Wavelength': '640', 'Power L': '60.00', 'Power R': '60.00'},
  {'Wavelength': '690', 'Power L': '20.00', 'Power R': '20.00'},
  {'Wavelength': '785', 'Power L': '20.00', 'Power R': '20.00'}],
 'tiles': [{'X': '440520',
   'Y': '762530',
   'Z': '3900',
   'Laser': '640',
   'Side': '0',
   'Exposure': '2',
   'Skip': '1',
   'Filter': '3',
   'NumImages': 1},
  {'X': '440520',
   'Y': '762530',
   'Z': '3900',
   'Laser': '561',
   'Side': '0',
   'Exposure': '2',
   'Skip': '1',
   'Filter': '2',
   'NumImages': 1},
  {'X': '472920',
   'Y': '762530',
   'Z': '3900',
   'Laser': '640',
   'Side': '1',
   'Exposure': '2',
   'Skip': '1',
   'Filter': '3',
   'NumImages': 1},
  {'X': '472920',
   'Y': '762530',
   'Z': '3900',
   'Laser': '561',
   'Side': '1',
   'Exposure': '2'